# How do I create an _app_ from a CWL JSON?
### Overview
This recipe is actually an _edge-case_, but one that the author uses frequently. It would also be a good fit for someone using code from a github repo, e.g. [Gaurav Kaushik's](https://github.com/gaurav-kaushik) excellent ones. We will:

 1. create a new project
 2. check that there are no apps
 3. upload our app
 4. check that it exists

### Prerequisites
 1. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 2. You have a **properly formatted** JSON file<sup>1</sup>.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You have at least one app in your project - one way to do this is by <a href="apps_copyFromPublicApps.ipynb" target="_blank">copying</a> one

<sup>1</sup> Note: Using _select all_; _copy_; _paste to text editor_; _save as JSON_ can fail due to slight inconsistencies in how the OS handles formatting. A much better approach is either (i) download a properly formatted JSON or (ii) use the _export_ button on the desired app on the Seven Bridges Platform and save the resulting webpage as a JSON directly. 

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The _Api_ object needs to know your **auth\_token** and the correct path. Here we assume you are using the .sbgrc file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify platform {cgc, sbpla, etc}
prof = 'sbpla'


config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Create a shiny, new project
To avoid any copy-errors with the app, we will make a new project. If this project name already exists, the code below will raise an interupt and fail. Be _creative_ with your project names, it's something you will look back on and laugh. 

#### PROTIPS
This next cell is more extensively detailed in this [recipe](projects_makeNew.ipynb)

In [ ]:
# [USER INPUT] Set project name and billing group index here:
new_project_name = 'Shiny and new'                          
index_billing = -1   


# Check if this project already exists. LIST all projects and check for name match
my_project = [p for p in api.projects.query(limit=100).all() \
              if p.name == new_project_name]      
              
if my_project:    # exploit fact that empty list is False, {list, tuple, etc} is True
    print('A project named {} exists, please choose a unique name'
          .format(new_project_name))
    raise KeyboardInterrupt
else:
    # Create a new project
    # What are my funding sources?
    billing_groups = api.billing_groups.query()  
    print((billing_groups[index_billing].name + \
           ' will be charged for computation and storage (if applicable)'))

    # Set up the information for your new project
    new_project = {
            'billing_group': billing_groups[index_billing].id,
            'description': """A project created by the API recipe (apps_installFromJSON).
                          This also supports **markdown**
                          _Pretty cool_, right?
                       """,
            'name': new_project_name
    }

    my_project = api.projects.create(
        name = new_project['name'], billing_group = new_project['billing_group'], 
        description = new_project['description']
    )
    
    # (re)list all projects, and get your new project
    my_project = [p for p in api.projects.query(limit=100).all() 
              if p.name == new_project_name][0]

    print('Your new project {} has been created.'.format(
        my_project.name))
    # Print description if it exists
    if hasattr(my_project, 'description'): 
        print('Project description: \n {}'.format(my_project.description)) 

## Sanity-check: do I have any apps?
Since you have just created the project, there will be **no** _Files_, _Apps_, or _Tasks_ in it. But just to be sure, let's query the apps in our project.

#### PROTIPS
This next cell is more extensively detailed in this [recipe](apps_listAll.ipynb)

In [ ]:
# List the apps in my project    
my_apps = api.apps.query(project = my_project.id, limit=100)
print('In project {}, you have {} apps.'.format(
    my_project.name, my_apps.total))

## Upload the JSON
We've pre-loaded on for you in the **/app_jsons/** folder. This should go _smoothly_. We use the **json** library to manage the formatting. Here you should choose an *app\_id* - dealer's choice on what you name it, but _please_ don't use the same ID twice. If you do, then Jack will be sad. Your call will also **fail** with _"Conflict: Requested app/revision already exists."_

Finally, we will re-check the apps in the project and see that it has uploaded sucessfully.

In [ ]:
# [USER INPUT] Specify the ID to assign to your app:
app_id = '/my-samtools-view'


#Load the Application JSONs
import json

f = open('app_jsons/samtools-view.json', 'r')
app_raw = f.read()
app = json.loads(app_raw)

# Create the Workflows
a_id = (my_project.id + app_id)
my_app_first = api.apps.install_app(id = a_id, raw = app)

# List the apps in my project    
my_apps = api.apps.query(project = my_project.id, limit=100)
print('In project {}, you have {} apps.'.format(
    my_project.name, my_apps.total))

for a in my_apps.all(): 
    print('App name is {}; \t App id: {}\n'.format(
        a.name, a.id))

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/add-an-app-using-raw-cwl)